In [2]:
import pandas as pd
from collections import defaultdict
import pyranges as pr
from collections import defaultdict
import numpy as np
import csv
import re
from itertools import combinations

/users/jferrer/jmidgley/annotation_project/lib64/python3.9/site-packages/sorted_nearest/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None) 

In [4]:
combined_df = pd.read_csv( '../data_processed/curated_HIT.tsv', sep='\t')

/tmp/19438950/ipykernel_1775176/558083981.py:1: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv( '../data_processed/curated_HIT.tsv', sep='\t')


In [5]:
# convert to GTF format
import ast

# Convert stringified exon lists to actual lists of tuples
def parse_exons(exon_str):
    if isinstance(exon_str, str):
        try:
            return ast.literal_eval(exon_str)
        except:
            return None 
    return exon_str  # already valid

combined_df["exons"] = combined_df["exons"].apply(parse_exons)

In [6]:
gtf_rows = []

for _, row in combined_df.iterrows():
    transcript_id = row["transcript_id"]
    gene_id = row.get("Gene stable ID", ".")
    gene_name = row.get("Gene name", ".")
    gene_type = row.get("Gene type", ".")
    chromosome = row["chr"]
    strand = row["Strand"]
    source = "custom"
    score = "."
    frame = "."

    exons = row.get("exons")

    # Skip missing or malformed exon lists
    if not isinstance(exons, list) or len(exons) == 0:
        continue

    for i, (start, end) in enumerate(exons, 1):
        if not isinstance((start, end), tuple) or len((start, end)) != 2:
            continue

        attributes = (
            f'gene_id "{gene_id}"; '
            f'transcript_id "{transcript_id}"; '
            f'gene_name "{gene_name}"; '
            f'gene_type "{gene_type}"; '
            #f'exon_number "{i}";'
        )

        gtf_rows.append([
            chromosome,
            source,
            "exon",
            start,
            end,
            score,
            strand,
            frame,
            attributes
        ])

# Convert to DataFrame
gtf_df = pd.DataFrame(gtf_rows, columns=[
    "chromosome", "source", "feature", "start", "end",
    "score", "strand", "frame", "attribute"
])
gtf_df

,chromosome,source,feature,start,end,score,strand,frame,attribute
0,chr1,custom,exon,61077274,61077628,.,+,.,"gene_id ""ENSG00000162599""; transcript_id ""ENCT00000006334.1""; gene_name ""NFIA""; gene_type ""protein_coding"";"
1,chr1,custom,exon,61077916,61079203,.,+,.,"gene_id ""ENSG00000162599""; transcript_id ""ENCT00000006334.1""; gene_name ""NFIA""; gene_type ""protein_coding"";"
2,chr1,custom,exon,160261734,160261922,.,+,.,"gene_id ""ENSG00000228606""; transcript_id ""ENCT00000013056.1""; gene_name ""DCAF8-DT""; gene_type ""lncRNA"";"
3,chr1,custom,exon,160281430,160281892,.,+,.,"gene_id ""ENSG00000228606""; transcript_id ""ENCT00000013056.1""; gene_name ""DCAF8-DT""; gene_type ""lncRNA"";"
4,chr1,custom,exon,239386568,239387227,.,+,.,"gene_id ""ENSG00000133019""; transcript_id ""ENCT00000019620.1""; gene_name ""CHRM3""; gene_type ""protein_coding"";"
...,...,...,...,...,...,...,...,...,...
3966117,chrY,custom,exon,12847239,12847327,.,+,.,"gene_id ""ENSG00000114374""; transcript_id ""STRT02296304""; gene_name ""USP9Y""; gene_type ""protein_coding"";"
3966118,chrY,custom,exon,12856340,12856496,.,+,.,"gene_id ""ENSG00000114374""; transcript_id ""STRT02296304""; gene_name ""USP9Y""; gene_type ""protein_coding"";"
3966119,chrY,custom,exon,12856633,12856845,.,+,.,"gene_id ""ENSG00000114374""; transcript_id ""STRT02296304""; gene_name ""USP9Y""; gene_type ""protein_coding"";"
3966120,chrY,custom,exon,12857566,12857661,.,+,.,"gene_id ""ENSG00000114374""; transcript_id ""STRT02296304""; gene_name ""USP9Y""; gene_type ""protein_coding"";"


In [7]:
# Write to file
gtf_df.to_csv("../data_processed/curated_HIT.gtf", sep="\t", index=False, header=False, quoting=csv.QUOTE_NONE)